In [1]:
%load_ext lab_black

In [2]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
df_btc = pd.read_csv("gemini_BTCUSD_day.csv", header=1)
df_eth = pd.read_csv("gemini_ETHUSD_day.csv", header=1)
df_sol = pd.read_csv("solana_investingcom.csv", header=0)

currency = pd.read_csv("FRB.csv")
sp500 = pd.read_csv("HistoricalData_spx.csv")
nasdaq = pd.read_csv("HistoricalData_comp.csv")

### preprocessing 

In [4]:
def preprocess_gemini_data(df):
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    df = df[["date", "symbol", "open", "high", "low", "close", "volume"]].copy()
    df.date = pd.to_datetime(df.date).dt.date
    df.index = df.date
    df = df.drop(columns="date")
    return df


def preprocess_solana(df):
    df.columns = df.columns.str.lower()
    df.date = pd.to_datetime(df.date)
    df.index = df.date
    return df


def preprocess_currency_data_to_series(currency, col_number):
    euro = currency.iloc[6:, [0, col_number]]
    euro.columns = ["date", "price"]
    euro.date = pd.to_datetime(euro.date, errors="coerce")
    euro.price = pd.to_numeric(euro.price, errors="coerce")
    euro.index = euro.date
    euro = euro.price.dropna()
    return euro


def preprocess_ns_data_to_series(df):
    s = df[["Date", "Open"]]
    s.columns = s.columns.str.lower()
    s.index = pd.to_datetime(s.date)
    s = s.open
    return s

In [5]:
df_btc = preprocess_gemini_data(df_btc)
df_eth = preprocess_gemini_data(df_eth)
df_sol = preprocess_solana(df_sol)

btc = df_btc.open
eth = df_eth.open
sol = df_sol.open

sp500 = preprocess_ns_data_to_series(sp500)
nasdaq = preprocess_ns_data_to_series(nasdaq)
currency = preprocess_currency_data_to_series(currency, 2)

In [6]:
ana = pd.concat(
    [
        btc.rename("btc"),
        eth.rename("eth"),
        sol.rename("sol"),
        sp500.rename("sp500"),
        nasdaq.rename("nas"),
        currency.rename("eur"),
    ],
    axis=1,
    join="outer",
)
# ana = ana.dropna(axis=0)[1:].copy()

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch/lib/python3.8/site-packages/pandas/core/indexes/base.py:3512: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  indexer = self._engine.get_indexer(target._get_engine_target())
/opt/homebrew/Caskroom/miniforge/base/envs/pytorch/lib/python3.8/site-packages/pandas/core/algorithms.py:1799: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  sorter = values.argsort()
/opt/homebrew/Caskroom/miniforge/base/envs/pytorch/lib/python3.8/site-packages/pandas/core/indexes/base.py:1963: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in orde

In [7]:
ana.corr()

,btc,eth,sol,sp500,nas,eur
btc,1.000000,0.927907,0.649573,0.900013,0.904599,0.346107
eth,0.927907,1.000000,0.899620,0.863732,0.836692,0.314734
sol,0.649573,0.899620,1.000000,0.809123,0.775379,-0.703472
sp500,0.900013,0.863732,0.809123,1.000000,0.986500,0.255733
nas,0.904599,0.836692,0.775379,0.986500,1.000000,0.304599
eur,0.346107,0.314734,-0.703472,0.255733,0.304599,1.000000


### helper function

In [8]:
def same_category(_df, i_range, normalize=False):
    df = _df.copy()
    for c in df.columns:
        df[c] = df[c] - 1
        if normalize:
            df[c] = df[c] / (df[c] - 1).max()
        sign = df[c].apply(lambda x: 1 if x > 0 else -1)
        df[c] = (
            df[c].apply(lambda x: 0 if (x > i_range[0]) and (x < i_range[1]) else 1)
            * sign
        )
    return df

In [9]:
def cat_given_cat(cat_df, normalize=True):
    ls = []
    for c in cat_df.columns[1:]:
        for cat in cat_df[c].unique():
            base = cat_df.loc[cat_df[c] == cat]
            sensitivity = base.iloc[:, 0].value_counts().rename(f"{c}_eq_{cat}")
            if normalize:
                sensitivity = sensitivity / len(base)
            ls.append(sensitivity)
    result = pd.concat(ls, axis=1)
    return result

### daily return

In [10]:
def daily_comparrision(coin):
    comp = ana[[coin, "sp500", "nas"]].replace(0, np.nan)
    comp["month"] = (
        pd.Series(comp.index).dt.year * 100 + pd.Series(comp.index).dt.month
    ).values
    comp["year"] = (pd.Series(comp.index).dt.year).values
    comp = comp.dropna()
    for c in [coin, "sp500", "nas"]:
        comp[f"ret_1d_{c}"] = (
            (comp[c] / comp[c].shift(1))
            .replace([np.inf, -np.inf], np.nan)
            .fillna(
                1,
            )
        )
    return comp

In [11]:
comp = daily_comparrision("eth")
comp.iloc[:, -3:].corr()

,ret_1d_eth,ret_1d_sp500,ret_1d_nas
ret_1d_eth,1.000000,0.153559,0.117533
ret_1d_sp500,0.153559,1.000000,0.874343
ret_1d_nas,0.117533,0.874343,1.000000


In [12]:
(comp.iloc[:, -3:] - 1).mean()

ret_1d_eth      0.006292
ret_1d_sp500    0.000583
ret_1d_nas      0.000830
dtype: float64

In [13]:
cat_daily = same_category(comp.iloc[:, -3:], [-0.005, 0.005])

In [14]:
cat_given_cat(cat_daily)

,ret_1d_sp500_eq_0,ret_1d_sp500_eq_1,ret_1d_sp500_eq_-1,ret_1d_nas_eq_0,ret_1d_nas_eq_1,ret_1d_nas_eq_-1
-1,0.412121,0.370056,0.558140,0.392523,0.410901,0.525157
0,0.088485,0.093220,0.081395,0.084112,0.094340,0.088050
1,0.499394,0.536723,0.360465,0.523364,0.494759,0.386792


### monthly return

In [15]:
def monthly_comparrision(coin):
    comp = daily_comparrision(coin)
    comp_month = comp.groupby(by="month")[
        f"ret_1d_{coin}", "ret_1d_sp500", "ret_1d_nas"
    ].prod()
    comp_month.columns = [f"ret_1m_{coin}", "ret_1m_sp500", "ret_1m_nas"]
    return comp_month

In [16]:
comp_month = monthly_comparrision("eth")
comp_month.corr()

/var/folders/y2/kn_782d959q_kshjqj842tpc0000gn/T/ipykernel_16992/53174413.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  comp_month = comp.groupby(by="month")[


,ret_1m_eth,ret_1m_sp500,ret_1m_nas
ret_1m_eth,1.000000,0.180315,0.189941
ret_1m_sp500,0.180315,1.000000,0.936659
ret_1m_nas,0.189941,0.936659,1.000000


In [17]:
(comp_month - 1).mean()

ret_1m_eth      0.161459
ret_1m_sp500    0.012148
ret_1m_nas      0.016965
dtype: float64

In [18]:
cat_monthly = same_category(comp_month, [-0.01, 0.01])

In [19]:
cat_given_cat(cat_monthly)

,ret_1m_sp500_eq_1,ret_1m_sp500_eq_-1,ret_1m_sp500_eq_0,ret_1m_nas_eq_1,ret_1m_nas_eq_-1,ret_1m_nas_eq_0
-1,0.295455,0.642857,0.363636,0.302326,0.692308,0.307692
0,0.022727,NaN,NaN,NaN,NaN,0.076923
1,0.681818,0.357143,0.636364,0.697674,0.307692,0.615385


In [20]:
comp_month.var()

ret_1m_eth      0.244540
ret_1m_sp500    0.001751
ret_1m_nas      0.002402
dtype: float64

### yearly return

In [21]:
def yearly_comparrision(coin):
    comp = daily_comparrision(coin)
    comp_month = comp.groupby(by="year")[
        f"ret_1d_{coin}", "ret_1d_sp500", "ret_1d_nas"
    ].prod()
    comp_month.columns = [f"ret_1y_{coin}", "ret_1y_sp500", "ret_1y_nas"]
    return comp_month

In [22]:
comp_year = yearly_comparrision("eth")
comp_year.corr()

/var/folders/y2/kn_782d959q_kshjqj842tpc0000gn/T/ipykernel_16992/395322584.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  comp_month = comp.groupby(by="year")[


,ret_1y_eth,ret_1y_sp500,ret_1y_nas
ret_1y_eth,1.000000,0.243026,0.256558
ret_1y_sp500,0.243026,1.000000,0.832889
ret_1y_nas,0.256558,0.832889,1.000000


In [23]:
comp_year

,ret_1y_eth,ret_1y_sp500,ret_1y_nas
year,,,
2016,0.912393,1.091621,1.140758
2017,86.416862,1.194323,1.278013
2018,0.183252,0.929268,0.956406
2019,0.967983,1.286618,1.341261
2020,5.681002,1.161139,1.443824
2021,5.034140,1.279096,1.220999
2022,0.757934,0.936373,0.893360


In [24]:
comp_year - 1

,ret_1y_eth,ret_1y_sp500,ret_1y_nas
year,,,
2016,-0.087607,0.091621,0.140758
2017,85.416862,0.194323,0.278013
2018,-0.816748,-0.070732,-0.043594
2019,-0.032017,0.286618,0.341261
2020,4.681002,0.161139,0.443824
2021,4.034140,0.279096,0.220999
2022,-0.242066,-0.063627,-0.106640


In [25]:
(comp_year - 1).mean()

ret_1y_eth      13.279081
ret_1y_sp500     0.125491
ret_1y_nas       0.182089
dtype: float64

In [26]:
cat_yearly = same_category(comp_year, [-0.02, 0.05])
cat_yearly

,ret_1y_eth,ret_1y_sp500,ret_1y_nas
year,,,
2016,-1,1,1
2017,1,1,1
2018,-1,-1,-1
2019,-1,1,1
2020,1,1,1
2021,1,1,1
2022,-1,-1,-1


In [27]:
cat_given_cat(cat_yearly)

,ret_1y_sp500_eq_1,ret_1y_sp500_eq_-1,ret_1y_nas_eq_1,ret_1y_nas_eq_-1
-1,0.4,1.0,0.4,1.0
1,0.6,NaN,0.6,NaN


### yearly return for months (yearly returns evaluated each month)

In [28]:
comp_ym = comp_month.copy()
comp_ym = comp_ym.rolling(12).apply(lambda x: np.product(x)).dropna()
# comp_ym = comp_ym - 1

In [29]:
cat_ym = same_category(comp_ym, [-0.02, 0.05])
cat_ym

,ret_1m_eth,ret_1m_sp500,ret_1m_nas
month,,,
201704,1,1,1
201705,1,1,1
201706,1,1,1
201707,1,1,1
201708,1,1,1
201709,1,1,1
201710,1,1,1
201711,1,1,1
201712,1,1,1


In [30]:
cat_given_cat(cat_ym).round(2)

,ret_1m_sp500_eq_1,ret_1m_sp500_eq_0,ret_1m_sp500_eq_-1,ret_1m_nas_eq_1,ret_1m_nas_eq_-1,ret_1m_nas_eq_0
-1,0.21,0.75,1.0,0.24,1.0,1.0
1,0.79,0.25,NaN,0.76,NaN,NaN


### magnitude

In [31]:
yret = comp_year - 1
yret

,ret_1y_eth,ret_1y_sp500,ret_1y_nas
year,,,
2016,-0.087607,0.091621,0.140758
2017,85.416862,0.194323,0.278013
2018,-0.816748,-0.070732,-0.043594
2019,-0.032017,0.286618,0.341261
2020,4.681002,0.161139,0.443824
2021,4.034140,0.279096,0.220999
2022,-0.242066,-0.063627,-0.106640


In [33]:
(yret.ret_1y_eth / yret.ret_1y_sp500)

year
2016     -0.956188
2017    439.560864
2018     11.547015
2019     -0.111707
2020     29.049518
2021     14.454320
2022      3.804489
dtype: float64

In [34]:
(yret.ret_1y_eth / yret.ret_1y_nas)

year
2016     -0.622395
2017    307.240121
2018     18.735458
2019     -0.093820
2020     10.546981
2021     18.254136
2022      2.269942
dtype: float64